In [1]:
from pyspark.sql import SparkSession
import os
spark = SparkSession.builder.appName('weekly_spark').getOrCreate()
#spark

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from pyspark.sql.functions import length, col, count, expr, monotonically_increasing_id, lit

reviews_schema = StructType([
    StructField("App ID", IntegerType(), True),
    StructField("Review", StringType(), True),
    StructField("Voted Up", StringType(), True)
])

weekly_top_news_schema = StructType([
    StructField("appnews", StructType([
        StructField("appid", IntegerType()),
        StructField("newsitems", ArrayType(StructType([
            StructField("gid", StringType()),
            StructField("title", StringType()),
            StructField("url", StringType()),
            StructField("is_external_url", BooleanType()),
            StructField("author", StringType()),
            StructField("contents", StringType()),
            StructField("feedlabel", StringType()),
            StructField("date", IntegerType()),
            StructField("feedname", StringType()),
            StructField("feed_type", IntegerType()),
            StructField("appid", IntegerType()),
            StructField("tags", ArrayType(StringType()))
        ]))),
        StructField("count", IntegerType())
    ]))
])

top_sellers_schema = StructType([
    StructField("Rank", IntegerType(), True),
    StructField("Game Name", StringType(), True),
    StructField("Free to Play", IntegerType(), True),
    ])

top_sellers_appids_schema = StructType([
    StructField("App ID", IntegerType(), True),
])

news_schema = StructType([
    StructField("App ID", IntegerType(), True),
    StructField("News 1", StringType(), True),
    StructField("News 2", StringType(), True),
    StructField("News 3", StringType(), True),
    StructField("News 4", StringType(), True),
    StructField("News 5", StringType(), True),
    StructField("News 6", StringType(), True),
    StructField("News 7", StringType(), True),
    StructField("News 7", StringType(), True),
    StructField("News 8", StringType(), True),
    StructField("News 10", StringType(), True)
])


In [3]:
WEEKLY_DATA_PATH = r'../data/weekly_data/'
reviews_path = os.path.join(WEEKLY_DATA_PATH, 'reviews/')
FILE_DATE = None

try:
    csv_files = [f for f in os.listdir(reviews_path) if f.endswith('.csv')]
    if csv_files:
        csv_file = csv_files[0]
        FILE_DATE = csv_file[0].split('.')[0].split('_')[0]
        csv_file_path = os.path.join(reviews_path, csv_file)
        most_daily_played = spark.read.csv(csv_file_path, header=True, schema=reviews_schema)
    else:
        print("No CSV files found in the 'reviews_path' directory.")
except Exception as e:
    print("An error occurred while reading the CSV file:", e)

In [4]:
# Cleaning the data
most_daily_played = most_daily_played.na.drop(subset=["Review", "Voted Up", "App ID"])
most_daily_played = most_daily_played.filter(length(col("Review")) >= 2)

# Counting the number of positive and negative reviews
counted_reviews = most_daily_played.groupBy("App ID").pivot("Voted Up", ["pos", "neg"]).agg(count("*").alias("count"))

# Seprarating the positive and negative reviews
neg_reviews_df = most_daily_played.filter(most_daily_played["Voted Up"] == "neg")
pos_reviews_df = most_daily_played.filter(most_daily_played["Voted Up"] == "pos")

neg_reviews_df = neg_reviews_df.withColumn("FILE_DATE", lit(FILE_DATE))
pos_reviews_df = pos_reviews_df.withColumn("FILE_DATE", lit(FILE_DATE))
counted_reviews = counted_reviews.withColumn("FILE_DATE", lit(FILE_DATE))

In [5]:
#Top_sellers
WEEKLY_TOP_SELLERS_PATH = WEEKLY_DATA_PATH + r'top_sellers/'
files = os.listdir(WEEKLY_TOP_SELLERS_PATH)

FILE_DATE = None
try:
    csv_file1 = [f for f in files if f.endswith('weekly_top_sellers.csv')]
    FILE_DATE = csv_file1[0].split('.')[0].split('_')[0]
    top_sellers_games = spark.read.csv(
        WEEKLY_TOP_SELLERS_PATH + csv_file1[0],
        header=True,
        schema=top_sellers_schema  
    )
    csv_file2 = [f for f in files if f.endswith('weekly_top_sellers_appIds.csv')]
    FILE_DATE = csv_file2[0].split('.')[0].split('_')[0]
    top_sellers_appids = spark.read.csv(
        WEEKLY_TOP_SELLERS_PATH + csv_file2[0],
        header=True,
        schema=top_sellers_appids_schema  
    )
    top_sellers_appids = top_sellers_appids.withColumn(
    "Rank",(monotonically_increasing_id() + 1).cast("int"))
    top_sellers = top_sellers_games.join(top_sellers_appids, on=["Rank"], how="inner")
    top_sellers = top_sellers.withColumn("FILE_DATE", lit(FILE_DATE))
    #top_sellers.show()   

except Exception as e:
    print("An error occurred while reading the CSV file:", e)

In [ ]:
neg_reviews_path = r"../saved_data/weekly_data/neg_reviews"
pos_reviews_path = r"../saved_data/weekly_data/pos_reviews"
counted_reviews_path = r"../saved_data/weekly_data/counted_reviews"
top_sellers_path = r"../saved_data/weekly_data/top_sellers"

# Save the DataFrame as CSV
neg_reviews_df.write.format("csv").mode("overwrite").option("header", "true").save(neg_reviews_path)
pos_reviews_df.write.format("csv").mode("overwrite").option("header", "true").save(pos_reviews_path)
counted_reviews.write.format("csv").mode("overwrite").option("header", "true").save(counted_reviews_path)
top_sellers.write.format("csv").mode("overwrite").option("header", "true").save(top_sellers_path)

In [55]:
# need to work on news
import pandas as pd
import json

In [73]:

WEEKLY_NEWS_PATH = WEEKLY_DATA_PATH + r'news/'
files = os.listdir(WEEKLY_NEWS_PATH)

# Create an empty list to store DataFrames from each JSON file
dfs = []

try:
    for file in files:
        with open(WEEKLY_NEWS_PATH + file, 'r') as json_file:
            json_data = json.load(json_file)

            df = pd.DataFrame()  # Create a new DataFrame for each JSON file

            for i in range(len(json_data['appnews']['newsitems'])):
                column_name = f"contents_{i}" 
                value = json_data['appnews']['newsitems'][i]['contents']
                
                if isinstance(value, (list, dict)):
                    value = str(value)
                df[column_name] = [value]

            df['App ID'] = json_data['appnews']['appid']

            # Append the DataFrame to the list
            dfs.append(df)

except Exception as e:
    print("An error occurred while reading the JSON file:", e)

# Concatenate all DataFrames in the list into a single DataFrame
news_df = pd.concat(dfs, ignore_index=True)

# The 'news_df' DataFrame now contains data from all JSON files
print(news_df)


                                          contents_0  \
0  Valve looks to be teasing the imminent release...   
1  Note: This update is for the Steam Deck Beta a...   
2  Моддер Halk Hogan PL спустя всего две недели п...   
3  According to the voice of Astarion himself, th...   
4  Датамайнер под ником<strong> TheThiny </strong...   
5  <strong>Payday 3</strong> is finally here, but...   
6  Heroes of Arkesia, The newest Ark Pass arrives...   
7  Компания Neowiz Games опубликовал обращение, в...   
8  n increase in Destiny 2 error code encounters ...   
9  Take heed, Legends, night has fallen and with ...   

                                          contents_1  \
0  Office Adjusted the weight of the vending mach...   
1  <a href="https://steamdb.info/topsellers/2023W...   
2  <strong>Starfield</strong> can be a rather gor...   
3  <strong>Larian Studios </strong>сообщила о вып...   
4  Watch Hollow experience Mortal Kombat 1's stor...   
5  <strong>Payday 3</strong> has only just rele

In [74]:
news_df

,contents_0,contents_1,contents_2,contents_3,contents_4,contents_5,contents_6,contents_7,contents_8,contents_9,App ID
0,Valve looks to be teasing the imminent release...,Office Adjusted the weight of the vending mach...,Anubis Various clipping adjustments Rebalanced...,Various bug fixes and tweaks for all shipped m...,Mirage Fixed a pixel boost on pillars in B bom...,Ancient Various bug fixes and tweaks Various b...,Overpass Various bug fixes and tweaks Added cu...,Ancient Fixed CT spawn area shape in minimap O...,Inferno Various bug fixes and tweaks Players w...,Added Premier mode Added a visible CS Rating a...,730
1,Note: This update is for the Steam Deck Beta a...,"<a href=""https://steamdb.info/topsellers/2023W...",Note: This update is for the Steam Deck Beta a...,SteamOS 3.5 has just been released to the Prev...,Note: This update is for the Steam Deck Beta a...,We have just shipped an updated Steam Deck Cli...,Note: This update is for the Steam Deck Beta a...,"<a href=""https://steamdb.info/topsellers/2023W...",We have just shipped an updated Steam Deck Cli...,Note: This update is for the Steam Deck Beta a...,1675200
2,Моддер Halk Hogan PL спустя всего две недели п...,<strong>Starfield</strong> can be a rather gor...,<strong>Starfield </strong>is the biggest Beth...,Seamless travel between planets is one of the ...,<strong>Starfield</strong>&apos;s community ke...,"Bummed that your <a href=""https://www.pcgamer....",<strong>Starfield</strong> Steam reviews give ...,"The <a href=""https://www.pcgamer.com/starfield...","Bethesda <a href=""https://twitter.com/Starfiel...","fter all these years, Bethesda Game Studios ha...",1716740
3,"According to the voice of Astarion himself, th...",<strong>Larian Studios </strong>сообщила о вып...,If you've finished <strong>Baldur's Gate 3</st...,<strong>Baldur's Gate 3 patch notes</strong> a...,Larian Studios has released patch 3 for <a hre...,"<a href=""https://www.pcgamer.com/baldurs-gate-...","Hello everyone, Patch 3 is now live for Baldur...",The <strong>Baldur's Gate 3 patch notes</stron...,"It&apos;s about time&#x2014;<a href=""https://w...","The next patch for <a href=""https://www.pcgame...",1086940
4,Датамайнер под ником<strong> TheThiny </strong...,Watch Hollow experience Mortal Kombat 1's stor...,RESOLVED The performance issue some PC players...,"Многочисленные проблемы, наблюдающиеся в Switc...",19 сентября состоялся полноценный релиз <stron...,Спустя две недели доминирования <strong>Starfi...,"<a href=""https://steamdb.info/topsellers/2023W...","В сети появился видеоролик, в котором проводит...",With its steadfast love of imploding skulls an...,<strong>Mortal Kombat 1</strong> isn't even te...,1971870
5,"<strong>Payday 3</strong> is finally here, but...",<strong>Payday 3</strong> has only just releas...,<strong>What are the Payday 3 error codes?</st...,<strong>Payday 3</strong> Steam reviews are of...,"<strong>September 23, 2023:</strong> It seems ...",<strong>What are the best Payday 3 mods?</stro...,"<strong>Payday 3</strong> is finally here, and...","<a href=""https://steamdb.info/app/1272080/char...",As someone who played and very much enjoyed th...,<strong>What are the best Payday 3 weapons?</s...,1272080
6,"Heroes of Arkesia, The newest Ark Pass arrives...",https://images.ctfassets.net/umhrp0op95v1/S6BJ...,https://images.ctfassets.net/umhrp0op95v1/5Adj...,The <strong>Lost Ark</strong> September Update...,"Heroes of Arkesia, The September “Jump to the ...","Heroes of Arkesia, We’re excited to announce t...",<strong>Lost Ark</strong> is an aesthetically ...,"Heroes of Arkesia, Join us for the live finale...",https://images.ctfassets.net/umhrp0op95v1/7zcO...,"Heroes of Arkesia, We’re back with your host R...",1599340
7,"Компания Neowiz Games опубликовал обращение, в...",<strong>Who is in the Lies of P cast?</strong>...,"Hello citizens of Krat, It's Choi Ji Won, the ...",Соулслайк-экшен <strong>Lies of P </strong>по ...,<strong>Is Lies of P Steam Deck compatible? </...,Спус

In [75]:
# Define a function to remove "<strong>" from each cell
def remove_strong(cell):
    if "<strong>" in cell:
        return cell.replace("<strong>", "")
    if "</strong>" in cell:
        return cell.replace("</strong>", "")
    if cell.startswith("<a href="):
        cell = ""
    else:
        return cell

# Apply the function to each cell in the DataFrame (except "App ID" column)
news_df.iloc[:, :-1] = news_df.iloc[:, :-1].applymap(remove_strong)


In [76]:
news_df

,contents_0,contents_1,contents_2,contents_3,contents_4,contents_5,contents_6,contents_7,contents_8,contents_9,App ID
0,Valve looks to be teasing the imminent release...,Office Adjusted the weight of the vending mach...,Anubis Various clipping adjustments Rebalanced...,Various bug fixes and tweaks for all shipped m...,Mirage Fixed a pixel boost on pillars in B bom...,Ancient Various bug fixes and tweaks Various b...,Overpass Various bug fixes and tweaks Added cu...,Ancient Fixed CT spawn area shape in minimap O...,Inferno Various bug fixes and tweaks Players w...,Added Premier mode Added a visible CS Rating a...,730
1,Note: This update is for the Steam Deck Beta a...,None,Note: This update is for the Steam Deck Beta a...,SteamOS 3.5 has just been released to the Prev...,Note: This update is for the Steam Deck Beta a...,We have just shipped an updated Steam Deck Cli...,Note: This update is for the Steam Deck Beta a...,None,We have just shipped an updated Steam Deck Cli...,Note: This update is for the Steam Deck Beta a...,1675200
2,Моддер Halk Hogan PL спустя всего две недели п...,Starfield</strong> can be a rather gorgeous ga...,Starfield </strong>is the biggest Bethesda gam...,Seamless travel between planets is one of the ...,Starfield</strong>&apos;s community keeps gett...,"Bummed that your <a href=""https://www.pcgamer....",Starfield</strong> Steam reviews give a lower ...,"The <a href=""https://www.pcgamer.com/starfield...","Bethesda <a href=""https://twitter.com/Starfiel...","fter all these years, Bethesda Game Studios ha...",1716740
3,"According to the voice of Astarion himself, th...",Larian Studios </strong>сообщила о выпуске тре...,If you've finished Baldur's Gate 3</strong> an...,Baldur's Gate 3 patch notes</strong> are here ...,Larian Studios has released patch 3 for <a hre...,"<a href=""https://www.pcgamer.com/baldurs-gate-...","Hello everyone, Patch 3 is now live for Baldur...",The Baldur's Gate 3 patch notes</strong> for B...,"It&apos;s about time&#x2014;<a href=""https://w...","The next patch for <a href=""https://www.pcgame...",1086940
4,Датамайнер под ником TheThiny </strong>раскрыл...,Watch Hollow experience Mortal Kombat 1's stor...,RESOLVED The performance issue some PC players...,"Многочисленные проблемы, наблюдающиеся в Switc...",19 сентября состоялся полноценный релиз Mortal...,Спустя две недели доминирования Starfield</str...,None,"В сети появился видеоролик, в котором проводит...",With its steadfast love of imploding skulls an...,Mortal Kombat 1</strong> isn't even technicall...,1971870
5,"Payday 3</strong> is finally here, but the Sta...",Payday 3</strong> has only just released from ...,What are the Payday 3 error codes?</strong> No...,Payday 3</strong> Steam reviews are officially...,"September 23, 2023:</strong> It seems players ...",What are the best Payday 3 mods?</strong> We'r...,"Payday 3</strong> is finally here, and that me...",None,As someone who played and very much enjoyed th...,What are the best Payday 3 weapons?</strong> P...,1272080
6,"Heroes of Arkesia, The newest Ark Pass arrives...",https://images.ctfassets.net/umhrp0op95v1/S6BJ...,https://images.ctfassets.net/umhrp0op95v1/5Adj...,The Lost Ark</strong> September Update is here...,"Heroes of Arkesia, The September “Jump to the ...","Heroes of Arkesia, We’re excited to announce t...",Lost Ark</strong> is an aesthetically pleasing...,"Heroes of Arkesia, Join us for the live finale...",https://images.ctfassets.net/umhrp0op95v1/7zcO...,"Heroes of Arkesia, We’re back with your host R...",1599340
7,"Компания Neowiz Games опубликовал обращение, в...",Who is in the Lies of P cast?</strong> The cas...,"Hello citizens of Krat, It's Choi Ji Won, the ...",Соулслайк-экшен Lies of P </strong>по мотивам ...,Is Lies of P Steam Deck compatible? </strong> ...,Спустя две недели доминирования Starfield</str...,Hello from the Lies of P Team. We have receive...,"Lies of P </strong>has only just released, and...",None,"We're coming towards the end of Septem